# FCA calculation at Middle Layer Super Output Areas (MSOA) level using OSRM distance (30 miles threshold/exclude London)

In [1]:
from access import access, weights, datasets
import logging
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import mapclassify as mc
from legendgram import legendgram
import palettable.matplotlib as palmpl
import seaborn as sns
import numpy as np
from scipy import stats

# Data import

## Get LA——MSOA code data

In [2]:
LA_MSOA = pd.read_csv('LA_MSOA_code.csv')
LA_MSOA

,LA_Code_2020,LA_name_2020,MSOA Code
0,E06000001,Hartlepool,E02002483
1,E06000001,Hartlepool,E02002484
2,E06000001,Hartlepool,E02002485
3,E06000001,Hartlepool,E02002487
4,E06000001,Hartlepool,E02002488
...,...,...,...
6786,E09000033,Westminster,E02000979
6787,E09000033,Westminster,E02000980
6788,E09000033,Westminster,E02000981
6789,E09000033,Westminster,E02000982


## Get MSOA boundary data

In [3]:
MSOA=gpd.read_file('MSOA_Boundary_with_population_ExcludeLondon.gpkg')
MSOA = MSOA.to_crs('epsg:27700')

In [4]:
MSOA

,MSOA11CD,MSOA11NM,pop0_17,pop18over,geometry
0,E02000984,Bolton 001,1462,5998,"POLYGON ((372121.741 414318.582, 372147.184 41..."
1,E02000985,Bolton 002,1328,5968,"POLYGON ((372971.325 411456.076, 373104.966 41..."
2,E02000986,Bolton 003,1714,6758,"POLYGON ((372147.184 413616.095, 372503.949 41..."
3,E02000987,Bolton 004,1251,5891,"POLYGON ((363078.556 411480.529, 363041.920 41..."
4,E02000988,Bolton 005,2176,6749,"POLYGON ((371044.136 412457.779, 371567.570 41..."
...,...,...,...,...,...
5801,E02006922,Colchester 022,2699,11342,"MULTIPOLYGON (((601481.105 224815.326, 601485...."
5802,E02006926,Thurrock 020,2342,5982,"POLYGON ((561100.600 178929.667, 560928.979 17..."
5803,E02006932,Liverpool 060,511,18899,"POLYGON ((335757.632 390987.474, 335739.219 39..."
5804,E02006933,Liverpool 061,654,8174,"POLYGON ((335096.788 389638.891, 334715.024 38..."


## Get the population weighted centroid of MSOA (demand points)

In [5]:
MSOA_points=gpd.read_file('MSOA_Population_Weighted_Centroids_with_population_ExcludeLondon.gpkg')
MSOA_points = MSOA_points.to_crs('epsg:27700')
MSOA_points=MSOA_points.drop(MSOA_points[MSOA_points.msoa11cd=='E02003950'].index)
MSOA_points=MSOA_points.drop(MSOA_points[MSOA_points.msoa11cd=='E02006781'].index)

In [47]:
MSOA_points

,objectid,msoa11cd,pop0_17,pop18over,geometry
0,1,E02002536,2171,7556,MULTIPOINT (445582.345 524175.434)
1,2,E02002537,2286,6293,MULTIPOINT (446777.151 524256.841)
2,3,E02002534,833,4806,MULTIPOINT (461356.929 515118.900)
3,4,E02002535,1640,7615,MULTIPOINT (446117.027 525455.836)
4,5,E02002532,1248,5559,MULTIPOINT (461053.212 516175.379)
...,...,...,...,...,...
5803,6787,E02004669,1468,6161,MULTIPOINT (393469.114 227500.260)
5804,6788,E02006096,1451,5570,MULTIPOINT (332829.367 109219.836)
5805,6789,E02003088,1706,6910,MULTIPOINT (343276.340 158947.520)
5806,6790,E02006070,1893,6627,MULTIPOINT (331710.269 136880.200)


## read vaccination site point data (supply points)

In [6]:
site = gpd.read_file('vaccination_site_exludeLondon.gpkg')
site = site.to_crs('epsg:27700')

In [7]:
site

,index,supply_value,geometry
0,0,1,MULTIPOINT (527671.845 409625.582)
1,2,1,MULTIPOINT (433880.091 387238.877)
2,3,1,MULTIPOINT (546372.813 254986.453)
3,4,1,MULTIPOINT (338131.255 397000.897)
4,5,1,MULTIPOINT (404128.280 441270.539)
...,...,...,...
2428,2829,1,MULTIPOINT (301789.330 89707.897)
2429,2830,1,MULTIPOINT (361632.845 178155.149)
2430,2831,1,MULTIPOINT (297993.791 145504.134)
2431,2832,1,MULTIPOINT (166574.610 27864.985)


# Read in the driving distance from the point of demand to the point of supply as cost (Generated by driving_cost_calculation_OSRM.ipynb in data_raw folder)

In [8]:
distance_cost_df = pd.read_csv('distance_cost_final_OSRM.csv') 
distance_cost_df = distance_cost_df.rename(columns={"Distance in meter": "cost", "origin_id": "origin", "destination_id": "dest"})
distance_cost_df=distance_cost_df[distance_cost_df.cost!=0]
distance_cost_df

,cost,origin,dest
0,30537.9,E02002536,43
1,11274.8,E02002536,55
2,33557.4,E02002536,73
3,53938.0,E02002536,109
4,53938.0,E02002536,112
...,...,...,...
1119880,46796.6,E02006679,2223
1119881,63990.0,E02006679,2823
1119882,27216.1,E02006679,2826
1119883,49104.6,E02006679,2827


### Count the number of MSOAs that are lack of vaccination service

In [17]:
m = MSOA_with_supply[MSOA_with_supply['MSOA_with_supply']==False]
print('There are',m['MSOA11CD'].count(),'MSOAs that are lack of vaccination service setting 10 miles as the vaccination service radius threshold.')

There are 1474 MSOAs that are lack of vaccination service setting 10 miles as the vaccination service radius threshold.


# Create a dataframe for the accessibility calculation using the Driving distance from the point of demand to the point of supply as cost

In [39]:
MSOA_points

,objectid,msoa11cd,pop0_17,pop18over,geometry
0,1,E02002536,2171,7556,MULTIPOINT (445582.345 524175.434)
1,2,E02002537,2286,6293,MULTIPOINT (446777.151 524256.841)
2,3,E02002534,833,4806,MULTIPOINT (461356.929 515118.900)
3,4,E02002535,1640,7615,MULTIPOINT (446117.027 525455.836)
4,5,E02002532,1248,5559,MULTIPOINT (461053.212 516175.379)
...,...,...,...,...,...
5803,6787,E02004669,1468,6161,MULTIPOINT (393469.114 227500.260)
5804,6788,E02006096,1451,5570,MULTIPOINT (332829.367 109219.836)
5805,6789,E02003088,1706,6910,MULTIPOINT (343276.340 158947.520)
5806,6790,E02006070,1893,6627,MULTIPOINT (331710.269 136880.200)


In [9]:
fca = access(demand_df = MSOA_points,
           demand_index='msoa11cd',
           demand_value='pop18over',
           supply_df= site,
           supply_index= 'index',
           supply_value=['supply_value'],
           cost_df              = distance_cost_df,
           cost_origin          = 'origin',
           cost_dest            = 'dest',
           cost_name            = 'cost',
           neighbor_cost_df     = distance_cost_df,
           neighbor_cost_origin = 'origin',
           neighbor_cost_dest   = 'dest',
           neighbor_cost_name   = 'cost')

### 2SFCA

In [10]:
# Using 2SFCA method, 16093.44 meters (10 miles) is used as service threshold radius to calculate the accessibility
fca.two_stage_fca(name = "2sfca_10",max_cost = 16093.44)

,2sfca_10_supply_value
msoa11cd,
E02000984,0.000058
E02000985,0.000053
E02000986,0.000047
E02000987,0.000051
E02000988,0.000048
...,...
E02006922,0.000110
E02006926,0.000129
E02006932,0.000090


In [11]:
fca.two_stage_fca(name = "2sfca_15",max_cost = 24140.16)
fca.two_stage_fca(name = "2sfca_20",max_cost = 32186.88)
fca.two_stage_fca(name = "2sfca_25",max_cost = 40233.60)
fca.two_stage_fca(name = "2sfca_30",max_cost = 48280.32)

,2sfca_30_supply_value
msoa11cd,
E02000984,0.000070
E02000985,0.000067
E02000986,0.000071
E02000987,0.000072
E02000988,0.000072
...,...
E02006922,0.000105
E02006926,0.000114
E02006932,0.000053


### E2SFCA

#### Set distance decay weight using gaussian function.

In [12]:
# Define a gaussian weight. Here, set the σ to be 5364.48 (meters), which is one third of the threshold, such that we'll be at the 3σ level at 16093.44 meters.
# 用3σ原理解释参数的选择 (According to the the empirical rule, also referred to as the three-sigma rule,
# for a normal distribution, almost all observed data will fall within three standard deviations.因此当距离接近threshold时，weight趋近于0)
gaussian_10 = weights.gaussian(sigma =5364.48)
gaussian_15 = weights.gaussian(sigma =8064.72)
gaussian_20 = weights.gaussian(sigma =10728.96)
gaussian_25 = weights.gaussian(sigma =13411.20)
gaussian_30 = weights.gaussian(sigma =16093.44)

fca.enhanced_two_stage_fca(name = "E2sfca_10", weight_fn = gaussian_10)
fca.enhanced_two_stage_fca(name = "E2sfca_15", weight_fn = gaussian_15)
fca.enhanced_two_stage_fca(name = "E2sfca_20", weight_fn = gaussian_20)
fca.enhanced_two_stage_fca(name = "E2sfca_25", weight_fn = gaussian_25)
fca.enhanced_two_stage_fca(name = "E2sfca_30", weight_fn = gaussian_30)

,E2sfca_30_supply_value
msoa11cd,
E02000984,0.000063
E02000985,0.000061
E02000986,0.000066
E02000987,0.000046
E02000988,0.000069
...,...
E02006922,0.000116
E02006926,0.000092
E02006932,0.000079


### 3SFCA

In [13]:
fca.three_stage_fca(name = "3sfca_10", weight_fn = gaussian_10)

,3sfca_10_supply_value
msoa11cd,
E02000984,0.000049
E02000985,0.000054
E02000986,0.000061
E02000987,0.000043
E02000988,0.000066
...,...
E02006922,0.000111
E02006926,0.000052
E02006932,0.000103


In [14]:
fca.three_stage_fca(name = "3sfca_15", weight_fn = gaussian_15)

,3sfca_15_supply_value
msoa11cd,
E02000984,0.000052
E02000985,0.000054
E02000986,0.000057
E02000987,0.000045
E02000988,0.000060
...,...
E02006922,0.000112
E02006926,0.000106
E02006932,0.000079


In [15]:
fca.three_stage_fca(name = "3sfca_20", weight_fn = gaussian_20)

,3sfca_20_supply_value
msoa11cd,
E02000984,0.000054
E02000985,0.000055
E02000986,0.000055
E02000987,0.000052
E02000988,0.000056
...,...
E02006922,0.000110
E02006926,0.000113
E02006932,0.000074


In [16]:
fca.three_stage_fca(name = "3sfca_25", weight_fn = gaussian_25)

,3sfca_25_supply_value
msoa11cd,
E02000984,0.000054
E02000985,0.000055
E02000986,0.000054
E02000987,0.000053
E02000988,0.000055
...,...
E02006922,0.000104
E02006926,0.000098
E02006932,0.000071


In [17]:
fca.three_stage_fca(name = "3sfca_30", weight_fn = gaussian_30)

,3sfca_30_supply_value
msoa11cd,
E02000984,0.000054
E02000985,0.000055
E02000986,0.000054
E02000987,0.000051
E02000988,0.000055
...,...
E02006922,0.000098
E02006926,0.000079
E02006932,0.000069


## See the accessibility score result

In [64]:
fca.access_df.head()

,pop18over,2sfca_10_supply_value,2sfca_15_supply_value,2sfca_20_supply_value,2sfca_25_supply_value,2sfca_30_supply_value,E2sfca_10_supply_value,E2sfca_15_supply_value,E2sfca_20_supply_value,E2sfca_25_supply_value,E2sfca_30_supply_value,3sfca_10_supply_value,3sfca_15_supply_value,3sfca_20_supply_value,3sfca_25_supply_value,3sfca_30_supply_value
msoa11cd,,,,,,,,,,,,,,,,
E02000984,5998,0.000058,0.000053,0.000074,0.000074,0.000070,0.000044,0.000052,0.000056,0.000060,0.000063,0.000049,0.000052,0.000054,0.000054,0.000054
E02000985,5968,0.000053,0.000056,0.000070,0.000075,0.000067,0.000049,0.000053,0.000055,0.000058,0.000061,0.000054,0.000054,0.000055,0.000055,0.000055
E02000986,6758,0.000047,0.000077,0.000081,0.000077,0.000071,0.000053,0.000054,0.000058,0.000062,0.000066,0.000061,0.000057,0.000055,0.000054,0.000054
E02000987,5891,0.000051,0.000032,0.000050,0.000069,0.000072,0.000035,0.000040,0.000040,0.000042,0.000046,0.000043,0.000045,0.000052,0.000053,0.000051
E02000988,6749,0.000048,0.000081,0.000083,0.000078,0.000072,0.000063,0.000060,0.000062,0.000066,0.000069,0.000066,0.000060,0.000056,0.000055,0.000055


In [18]:
MSOA_fca = pd.merge(left=MSOA, right=fca.access_df, how='left', left_on='MSOA11CD', right_on='msoa11cd')
MSOA_fca=MSOA_fca.fillna(0)
MSOA_fca = MSOA_fca.rename(columns={"2sfca_10_supply_value": "2sfca_10", "2sfca_15_supply_value": "2sfca_15", 'pop18over_x':'pop18over',
                                    "2sfca_20_supply_value": "2sfca_20", "2sfca_25_supply_value": "2sfca_25","2sfca_30_supply_value": "2sfca_30",
                                   "E2sfca_10_supply_value": "E2sfca_10", "E2sfca_15_supply_value": "E2sfca_15", 
                                    "E2sfca_20_supply_value": "E2sfca_20", "E2sfca_25_supply_value": "E2sfca_25","E2sfca_30_supply_value": "E2sfca_30",
                                   "3sfca_10_supply_value": "3sfca_10", "3sfca_15_supply_value": "3sfca_15", 
                                    "3sfca_20_supply_value": "3sfca_20", "3sfca_25_supply_value": "3sfca_25","3sfca_30_supply_value": "3sfca_30"})

In [19]:
MSOA_fca.head()

,MSOA11CD,MSOA11NM,pop0_17,pop18over,geometry,pop18over_y,2sfca_10,2sfca_15,2sfca_20,2sfca_25,...,E2sfca_10,E2sfca_15,E2sfca_20,E2sfca_25,E2sfca_30,3sfca_10,3sfca_15,3sfca_20,3sfca_25,3sfca_30
0,E02000984,Bolton 001,1462,5998,"POLYGON ((372121.741 414318.582, 372147.184 41...",5998,0.000058,0.000053,0.000074,0.000074,...,0.000044,0.000052,0.000056,0.000060,0.000063,0.000049,0.000052,0.000054,0.000054,0.000054
1,E02000985,Bolton 002,1328,5968,"POLYGON ((372971.325 411456.076, 373104.966 41...",5968,0.000053,0.000056,0.000070,0.000075,...,0.000049,0.000053,0.000055,0.000058,0.000061,0.000054,0.000054,0.000055,0.000055,0.000055
2,E02000986,Bolton 003,1714,6758,"POLYGON ((372147.184 413616.095, 372503.949 41...",6758,0.000047,0.000077,0.000081,0.000077,...,0.000053,0.000054,0.000058,0.000062,0.000066,0.000061,0.000057,0.000055,0.000054,0.000054
3,E02000987,Bolton 004,1251,5891,"POLYGON ((363078.556 411480.529, 363041.920 41...",5891,0.000051,0.000032,0.000050,0.000069,...,0.000035,0.000040,0.000040,0.000042,0.000046,0.000043,0.000045,0.000052,0.000053,0.000051
4,E02000988,Bolton 005,2176,6749,"POLYGON ((371044.136 412457.779, 371567.570 41...",6749,0.000048,0.000081,0.000083,0.000078,...,0.000063,0.000060,0.000062,0.000066,0.000069,0.000066,0.000060,0.000056,0.000055,0.000055


## Add vaccination population data and calculate vaccination rate

In [20]:
# Read the vaccination population data
vaccination_pop = pd.read_csv('vaccination_pop_2021_11_18.csv')

In [22]:
vaccination_pop['18over_2dose_total']=vaccination_pop['18over1st_dose']+vaccination_pop['18over2nd_dose']

In [23]:
# merge the population data
MSOA_fca = pd.merge(left = MSOA_fca, right=vaccination_pop, how='left', left_on='MSOA11CD', right_on='MSOA')

# merge the LA information
MSOA_fca = pd.merge(left = MSOA_fca, right=LA_MSOA, how='left', left_on='MSOA11CD', right_on='MSOA Code')

In [24]:
MSOA_fca['vaccination_percentage_1stdose'] = MSOA_fca['18over1st_dose']/MSOA_fca['pop18over']
MSOA_fca['vaccination_percentage_2nddose'] = MSOA_fca['18over2nd_dose']/MSOA_fca['pop18over']
MSOA_fca['vaccination_percentage_total'] = (MSOA_fca['18over1st_dose']+MSOA_fca['18over2nd_dose'])/MSOA_fca['pop18over']/2

# Set vaccination rate greater than 1 to 1
MSOA_fca['vaccination_percentage_2nddose']= np.where(MSOA_fca['vaccination_percentage_2nddose']>1, 1, MSOA_fca['vaccination_percentage_2nddose'])
MSOA_fca['vaccination_percentage_1stdose']= np.where(MSOA_fca['vaccination_percentage_1stdose']>1, 1, MSOA_fca['vaccination_percentage_1stdose'])
MSOA_fca['vaccination_percentage_total']= np.where(MSOA_fca['vaccination_percentage_total']>1, 1, MSOA_fca['vaccination_percentage_total'])

In [94]:
MSOA_fca.columns

Index(['MSOA11CD', 'MSOA11NM', 'pop0_17', 'pop18over', 'geometry',
       'pop18over_y', '2sfca_10', '2sfca_15', '2sfca_20', '2sfca_25',
       '2sfca_30', 'E2sfca_10', 'E2sfca_15', 'E2sfca_20', 'E2sfca_25',
       'E2sfca_30', '3sfca_10', '3sfca_15', '3sfca_20', '3sfca_25', '3sfca_30',
       'MSOA', '18over1st_dose', '18over2nd_dose', '18over_2dose_total',
       'LA_Code_2020', 'LA_name_2020', 'MSOA Code',
       'vaccination_percentage_1stdose', 'vaccination_percentage_2nddose',
       'vaccination_percentage_total'],
      dtype='object')

In [25]:
MSOA_fca.head(1)

,MSOA11CD,MSOA11NM,pop0_17,pop18over,geometry,pop18over_y,2sfca_10,2sfca_15,2sfca_20,2sfca_25,...,MSOA,18over1st_dose,18over2nd_dose,18over_2dose_total,LA_Code_2020,LA_name_2020,MSOA Code,vaccination_percentage_1stdose,vaccination_percentage_2nddose,vaccination_percentage_total
0,E02000984,Bolton 001,1462,5998,"POLYGON ((372121.741 414318.582, 372147.184 41...",5998,0.000058,0.000053,0.000074,0.000074,...,E02000984,5896,5734,11630,E08000001,Bolton,E02000984,0.982994,0.955985,0.96949


In [27]:
# Export fca result (exclude London region)
MSOA_fca_exclude = pd.DataFrame()
MSOA_fca_exclude =MSOA_fca
MSOA_fca_exclude = MSOA_fca_exclude.drop(['MSOA11NM','MSOA','pop18over_y','LA_Code_2020','LA_name_2020','MSOA Code'], axis=1)
#MSOA_fca_exclude
MSOA_fca_exclude.to_csv('fca_result_before_norm_exclude_London.csv',index=False)
MSOA_fca_exclude.to_file("fca_result_before_norm_exclude_London.gpkg", driver="GPKG")